In [287]:
import numpy as np
from transformers import BertModel, BertTokenizerFast
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import random
import scipy.stats as sc
import time

In [276]:
train_set = ["生活不易，要好好珍惜。",
            "学无止境，要不断进步。",
            "人生短暂，要活得有意义。",
            "爱与被爱，是人类最美好的情感。",
            "勤奋和毅力，是成功的关键。",
            "没有付出就没有收获。",
            "健康是最大的财富。",
            "时光荏苒，珍惜眼前人。",
            "未来可期，要有信心。",
            "诚信为本，处事公正。",
            "人与人之间要互相尊重。",
            "团结就是力量。",
            "金钱不能代替一切。",
            "知足常乐，心态很重要。",
            "人生有苦有乐，要看淡一些。",
            "热爱生活，享受每一天。",
            "科技在不断进步，我们也要不断学习。",
            "幸福就在身边，要学会发现。",
            "人生就像一场旅行，要不断探索。",
            "梦想不只是梦，要用行动去实现。",
            "生命不息，奋斗不止。",
            "不忘初心，方得始终。",
            "世上无难事，只怕有心人。",
            "好的开始是成功的一半。",
            "沉默不是金，适时表达很重要。",
            "勇敢面对困难，你会更强大。",
            "机会只留给有准备的人。",
            "爱情需要用心经营。",
            "有志者事竟成。",
            "做事要有计划和步骤。",
            "改变自己，改变世界。",
            "坚持就是胜利。",
            "珍惜友情，朋友很重要。",
            "年轻就是资本，要好好利用。",
            "世界上没有偶然，只有必然。",
            "勇于创新，不断超越自己。",
            "现在努力，未来才会更美好。",
            "难以忘怀的经历，是人生中的财富。",
            "宽容和理解，让世界更美好。",
            "热爱生命，就会更有活力。",
            "坚定信念，无所不能。",
            "路漫漫其修远兮，吾将上下而求索。",
            "付出不一定有回报，但不付出一定无回报。",
            "人生没有彩排，每一天都是现场直播"]

In [277]:
modelname = 'bert-base-uncased'
tokenizer = BertTokenizerFast.from_pretrained(modelname)
bertmodel = BertModel.from_pretrained(modelname)
input = tokenizer(train_set, padding=True,
            return_tensors="pt", return_attention_mask=True)
output = bertmodel(**input)
train_data = output["last_hidden_state"].sum(axis=1)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([44, 768])


In [278]:

gpu = torch.cuda.is_available()
device = torch.device("cuda:0" if gpu else "cpu")
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(768, 512)	
        self.fc2 = nn.Linear(512, 256)	
        self.fc31 = nn.Linear(256, 3)
        self.fc32 = nn.Linear(256, 3)
        self.fc3 = nn.Linear(3, 256)
        self.fc4 = nn.Linear(256, 512)
        self.fc5 = nn.Linear(512, 768)

    def encode(self, x):
        x = F.relu(self.fc1(x))
        h1 = F.relu(self.fc2(x))
        return self.fc31(h1), self.fc32(h1)

    def reparameterize(self, mu, logvar):
        dis = torch.distributions.Normal(mu, torch.exp(0.5*logvar))
        return dis.rsample()

    def decode(self, z):
        z = F.relu(self.fc3(z))
        h3 = F.relu(self.fc4(z))
        return torch.sigmoid(self.fc5(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 768))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


model = VAE()
optimizer = optim.Adam(model.parameters(), lr=1e-3)


def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 768), reduction='sum')

    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD


In [279]:
train_dataset = torch.utils.data.DataLoader(train_data.detach(),batch_size=11, shuffle=True)
for data in train_dataset:
    data = data.to(device)
    optimizer.zero_grad()
    recon_batch, mu, logvar = model(data)
    loss = loss_function(recon_batch, data, mu, logvar)
    loss.backward()
    optimizer.step()
    print("loss : ", loss.item())

loss :  5864.63623046875
loss :  -26096.232421875
loss :  -848900.9375
loss :  -167888.484375


In [284]:
class SIXYAO():
    def __init__(self,model) -> None:
        self.GUA = {0:"阳",1:"阴" ,2:"阳" ,3:"阴"}
        self.BIANGUA = {0:"阴",1:"阴" ,2:"阳" ,3:"阳"}
        self.BAGUA = {"阴阴阴":"坤","阴阴阳":"艮","阴阳阴":"坎","阴阳阳":"巽","阳阴阴":"震","阳阴阳":"离","阳阳阴":"兑","阳阳阳":"乾"}
        self.ben = None
        self.bian = None
        self.model = model

    def bu(self, string):
        modelname = 'bert-base-uncased'
        tokenizer = BertTokenizerFast.from_pretrained(modelname)
        bertmodel = BertModel.from_pretrained(modelname)
        input = tokenizer(string, padding=True,
                    return_tensors="pt", return_attention_mask=True)
        output = bertmodel(**input)
        data = output["last_hidden_state"].sum(axis=1)
        collection = []
        with torch.no_grad():
            mu, logvar = self.model.encode(data)
            dis = sc.norm(mu, torch.exp(0.5*logvar))
            z = self.model.reparameterize(mu, logvar)
        coins  = torch.distributions.Bernoulli(torch.tensor(dis.cdf(z)))
        for i in range(6):
            coin = coins.sample().squeeze().numpy()
            collection.append(coin)
        collection = np.sum(collection, axis=1)
        self.ben = [GUA[i] for i in collection]
        self.bian = [BIANGUA[i] for i in collection]

    def __solver(self, input):
        output = []
        for i in range(4):
            index = ""
            for j in input[i:i+3]:
                index += j
            output.append(self.BAGUA[index])
        return output

    def jie(self):
        assert type(self.ben) != type(None), print("你需要先补一卦")
        print("本卦 :", self.__solver(self.ben))
        print("变卦 :", self.__solver(self.bian))

In [295]:
torch.manual_seed(time.time())
sixyao = SIXYAO(model)
sixyao.bu("赛博占卜")
sixyao.jie()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


本卦 : ['乾', '兑', '震', '坤']
变卦 : ['乾', '乾', '兑', '震']
